In [7]:
import cv2
import numpy as np

# Function to process each frame and find the drop edge
def process_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        cv2.drawContours(frame, [largest_contour], -1, (0, 255, 0), 2)

        # Get the height of the drop edge
        y_coords = largest_contour[:, :, 1]
        drop_height = frame.shape[0] - np.min(y_coords)  # Calculate height from the bottom of the frame

        return drop_height
    return None

def main(video_path):
    cap = cv2.VideoCapture(video_path)
    previous_height = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Process the current frame and get the drop height
        drop_height = process_frame(frame)

        if drop_height is not None:
            # Display the drop height on the frame
            cv2.putText(frame, f'Drop Height: {drop_height}', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

            # Calculate and display the change in height
            if previous_height is not None:
                change = drop_height - previous_height
                cv2.putText(frame, f'Change: {change}', (10, 70),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            previous_height = drop_height

        cv2.imshow('Drop Edge Tracking', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # Specify the path to your video file
    video_path = 'path/to/your/video.mp4'
    main(video_path)